In [1]:
import pandas as pd
from sqlalchemy import create_engine


import pymysql.cursors
import pymysql
import codecs
import os

import joblib 
import numpy as np
from flask import Flask
import flask

import random
import math
# from sklearn.externals import joblib
 

In [2]:
# Connect to the database
conn = pymysql.connect(host = 'mysql',
                       user = 'root',
                       password = 'iii',
                       charset = 'utf8mb4',
                       cursorclass = pymysql.cursors.DictCursor)

In [3]:
### 依照使用者的分群偏好 對餐廳的標籤進行權重的計算
### 以計算所得的分數以及相關的條件 (CP值等) 排序

In [4]:
profile_mean_csv = pd.read_csv('profile_mean.csv')
profile_mean_csv = profile_mean_csv.rename(columns = {'df$store_nm_en': 'store_nm'})

profile_mean = profile_mean_csv.loc[:, ~profile_mean_csv.columns.str.contains('^Unnamed')]
 
profile_mean

,label_code,label_desc,請問您今年幾歲呢?,請問您去過東京幾次呢?,請問您上一次去東京幾天?,問您上一次去東京的總預算為新台幣多少錢?,潮男,靚女
0,0,年紀大 男性 花費高 有錢人 東京老司機 有家庭比例相對高,51.375000,16.500000,5.875000,105062.50000,1,0
1,1,青壯 女性 花費適中,36.250000,4.029412,5.617647,51647.05882,0,1
2,2,青壯 男性 花費適中,33.618182,3.018182,6.381818,61272.72727,1,0


In [5]:
profile_mean.columns = ['label_code',
                   'label_desc',
                   '年齡',
                   '旅遊次數',
                   '旅遊天數',
                   '總預算',
                   '潮男',
                   '靚女'
                   ]
profile_mean

,label_code,label_desc,年齡,旅遊次數,旅遊天數,總預算,潮男,靚女
0,0,年紀大 男性 花費高 有錢人 東京老司機 有家庭比例相對高,51.375000,16.500000,5.875000,105062.50000,1,0
1,1,青壯 女性 花費適中,36.250000,4.029412,5.617647,51647.05882,0,1
2,2,青壯 男性 花費適中,33.618182,3.018182,6.381818,61272.72727,1,0


In [6]:
profile_food_mean = pd.read_csv('profile_food_mean.csv')

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

food_mean = profile_food_mean.loc[:, ~profile_food_mean.columns.str.contains('^Unnamed')]
food_mean

,label,請問您在東京旅遊時每餐的預算約為多少?(新台幣/人均),關於出國旅行那些事，您的看法如何呢？ - 都出國旅行了，我會願意花錢在美食上,請問您最喜歡日本甚麼類型的食物?(至多五名) - 壽司或生魚片,請問您最喜歡日本甚麼類型的食物?(至多五名) - 天婦羅炸雞或炸豬排,請問您最喜歡日本甚麼類型的食物?(至多五名) - 壽喜燒或涮涮鍋,請問您最喜歡日本甚麼類型的食物?(至多五名) - 拉麵或日式炒麵,請問您最喜歡日本甚麼類型的食物?(至多五名) - 和牛 (和牛)或河豚（河豚）或懷石料理（懐石料理）,請問您最喜歡日本甚麼類型的食物?(至多五名) - 咖哩飯或牛肉蓋飯,請問您最喜歡日本甚麼類型的食物?(至多五名) - 蕎麥麵或烏龍麵,請問您最喜歡日本甚麼類型的食物?(至多五名) - 日式蒲燒鰻‎或烤肉,請問您最喜歡日本甚麼類型的食物?(至多五名) - 甜點,請問您最喜歡日本甚麼類型的食物?(至多五名) - 其他,餐廳知名度(網路評價、店家氣氛或服務態度等),餐廳交通便利性,CP值高,食物類型
0,0,19671.875000,4.3750,0.687500,0.312500,0.3125,0.687500,0.687500,0.2500,0.4375,0.562500,0.2500,0.0000,2.1875,1.812500,1.9375,2.812500
1,1,886.470588,4.2537,0.647100,0.397100,0.2794,0.661800,0.544118,0.2059,0.2941,0.308824,0.4853,0.0441,2.6176,2.073529,2.6324,2.676500
2,2,1326.363636,4.3333,0.763636,0.436364,0.2545,0.818182,0.563600,0.3091,0.1636,0.418182,0.2545,0.0000,2.4909,1.818200,2.5818,2.927273


In [7]:
food_mean.columns = ['label', 
                        '每餐預算(新台幣/人均)',
                        '願意花錢在美食上',
                        '壽司或生魚片',
                        '天婦羅炸雞或炸豬排',
                        '壽喜燒或涮涮鍋',
                        '拉麵或日式炒麵',
                        '和牛或河豚或懷石料理',
                        '咖哩飯或牛肉蓋飯',
                        '蕎麥麵或烏龍麵',
                        '日式蒲燒鰻或烤肉',
                        '甜點',
                        '其他',
                        '餐廳知名度(網路評價、店家氣氛或服務態度等)',
                        '餐廳交通便利性',
                        'CP值高',
                        '食物類型'
                       ]
food_mean

,label,每餐預算(新台幣/人均),願意花錢在美食上,壽司或生魚片,天婦羅炸雞或炸豬排,壽喜燒或涮涮鍋,拉麵或日式炒麵,和牛或河豚或懷石料理,咖哩飯或牛肉蓋飯,蕎麥麵或烏龍麵,日式蒲燒鰻或烤肉,甜點,其他,餐廳知名度(網路評價、店家氣氛或服務態度等),餐廳交通便利性,CP值高,食物類型
0,0,19671.875000,4.3750,0.687500,0.312500,0.3125,0.687500,0.687500,0.2500,0.4375,0.562500,0.2500,0.0000,2.1875,1.812500,1.9375,2.812500
1,1,886.470588,4.2537,0.647100,0.397100,0.2794,0.661800,0.544118,0.2059,0.2941,0.308824,0.4853,0.0441,2.6176,2.073529,2.6324,2.676500
2,2,1326.363636,4.3333,0.763636,0.436364,0.2545,0.818182,0.563600,0.3091,0.1636,0.418182,0.2545,0.0000,2.4909,1.818200,2.5818,2.927273


In [8]:
# s=0.687500+0.312500+0.3125+0.687500+0.687500+0.2500+0.4375+0.562500+0.2500+0.0000
# 0.687500/s

In [9]:
# 計算公式
# if food類別 A:
#    (food類別 A / sum(food類別 A mean + food類別 B mean + food類別 C mean ... + food類別 K mean) ) x 旅遊天數 (by 午餐 / 晚餐)

# food類別 A: 壽司或生魚片
# food類別 B: 天婦羅炸雞或炸豬排
# food類別 C: 壽喜燒或涮涮鍋
# food類別 D: 拉麵或日式炒麵
# food類別 E: 和牛或河豚或懷石料理
# food類別 F: 和牛或河豚或懷石料理
# food類別 G: 咖哩飯或牛肉蓋飯
# food類別 H: 蕎麥麵或烏龍麵
# food類別 I: 日式蒲燒鰻或烤肉
# food類別 J: 甜點
# food類別 K: 其他

In [10]:
# for test
# clf = joblib.load('food_clf.joblib')

# # 年齡, 旅遊次數, 旅遊天數 , 總預算 ,性別(0:男  1:女), 一餐預算
# userdata = [20, 1, 6, 3000000, 1, 10000] #  cluster: 1

# # 分群 cluster
# cls = clf.predict(np.array(userdata).reshape((1,-1)))[0]

# print('cluster:\n', cls, '\n')

# print('userdata[0]: ', userdata[0])
# print('userdata[1]: ', userdata[1])
# print('userdata[2]: ', userdata[2])
# print('userdata[3]: ', userdata[3])
# print('userdata[4]: ', userdata[4])
# print('userdata[5]: ', userdata[5])

In [11]:
# 計算各個 food_catg 的佔比
# 計算公式
# if food類別 A:
#    (food類別 A / sum(food類別 A mean + food類別 B mean + food類別 C mean ... + food類別 K mean) ) x 旅遊天數 (by 午餐 / 晚餐)
# cls = 2
# sum_catg =  (food_mean.loc[cls, '壽司或生魚片'] +          # 1
#              food_mean.loc[cls, '天婦羅炸雞或炸豬排'] +    # 2
#              food_mean.loc[cls, '壽喜燒或涮涮鍋'] +        # 3
#              food_mean.loc[cls, '拉麵或日式炒麵'] +        # 4
#              food_mean.loc[cls, '和牛或河豚或懷石料理'] +  # 5
#              food_mean.loc[cls, '咖哩飯或牛肉蓋飯'] +     # 6
#              food_mean.loc[cls, '蕎麥麵或烏龍麵'] +       # 7
#              food_mean.loc[cls, '日式蒲燒鰻或烤肉'] +     # 8
#              food_mean.loc[cls, '其他'])                 # 9   
    

# food_catg1 = round((food_mean.loc[cls, '壽司或生魚片'] / sum_catg), 2)
# food_catg2 = round((food_mean.loc[cls, '天婦羅炸雞或炸豬排'] / sum_catg), 2)
# food_catg3 = round((food_mean.loc[cls, '壽喜燒或涮涮鍋'] / sum_catg), 2)
# food_catg4 = round((food_mean.loc[cls, '拉麵或日式炒麵'] / sum_catg), 2)
# food_catg5 = round((food_mean.loc[cls, '和牛或河豚或懷石料理'] / sum_catg), 2)
# food_catg6 = round((food_mean.loc[cls, '咖哩飯或牛肉蓋飯'] / sum_catg), 2)
# food_catg7 = round((food_mean.loc[cls, '蕎麥麵或烏龍麵'] / sum_catg), 2)
# food_catg8 = round((food_mean.loc[cls, '日式蒲燒鰻或烤肉'] / sum_catg), 2)
# food_catg9 = round((food_mean.loc[cls, '其他'] / sum_catg), 2)

# food_catg1 = (food_mean.loc[cls, '壽司或生魚片'] / sum_catg) * 100
# food_catg2 = (food_mean.loc[cls, '天婦羅炸雞或炸豬排'] / sum_catg) * 100
# food_catg3 = (food_mean.loc[cls, '壽喜燒或涮涮鍋'] / sum_catg) * 100
# food_catg4 = (food_mean.loc[cls, '拉麵或日式炒麵'] / sum_catg) * 100
# food_catg5 = (food_mean.loc[cls, '和牛或河豚或懷石料理'] / sum_catg) * 100
# food_catg6 = (food_mean.loc[cls, '咖哩飯或牛肉蓋飯'] / sum_catg) * 100
# food_catg7 = (food_mean.loc[cls, '蕎麥麵或烏龍麵'] / sum_catg) * 100
# food_catg8 = (food_mean.loc[cls, '日式蒲燒鰻或烤肉'] / sum_catg) * 100
# food_catg9 = (food_mean.loc[cls, '其他'] / sum_catg) * 100

In [12]:
# print('第', cls, '群 - 各食物類別之佔比:')
# print('food_catg１: ', food_catg1)
# print('food_catg２: ', food_catg2)
# print('food_catg３: ', food_catg3)
# print('food_catg４: ', food_catg4)
# print('food_catg５: ', food_catg5)
# print('food_catg６: ', food_catg6)
# print('food_catg７: ', food_catg7)
# print('food_catg８: ', food_catg8)
# print('food_catg９: ', food_catg9)
# print('sum(總佔比): ', food_catg1 + food_catg2 + food_catg3 + food_catg4 + food_catg5 + food_catg6 + food_catg7 + food_catg8 + food_catg9)

In [13]:
# distribution_0 = {'壽司或生魚片':0.1746031746031746, '天婦羅炸雞或炸豬排':0.07936507936507936, 
#                   '壽喜燒或涮涮鍋':0.07936507936507936, '拉麵或日式炒麵':0.1746031746031746, 
#                   '和牛或河豚或懷石料理':0.1746031746031746, '咖哩飯或牛肉蓋飯':0.06349206349206349, 
#                   '蕎麥麵或烏龍麵':0.1111111111111111, '日式蒲燒鰻或烤肉':0.14285714285714285, 
#                   '其他':0.0}

# distribution_1 ={'壽司或生魚片':0.19131148442456664, '天婦羅炸雞或炸豬排':0.1174003870576347, 
#                   '壽喜燒或涮涮鍋':0.1608654339084011, '拉麵或日式炒麵':0.19565745694974226, 
#                   '和牛或河豚或懷石料理':0.1746031746031746, '咖哩飯或牛肉蓋飯':0.06087317979140515, 
#                   '蕎麥麵或烏龍麵':0.08694901494245873, '日式蒲燒鰻或烤肉':0.09130208293298156, 
#                   '其他':0.013037917575526796}

# distribution_2 = {'壽司或生魚片':0.20488392783360218, '天婦羅炸雞或炸豬排':0.11707668350520666, 
#                   '壽喜燒或涮涮鍋':0.06828247965477237, '拉麵或日式炒麵':0.21951864742200775, 
#                   '和牛或河豚或懷石料理':0.15121416712546054, '咖哩飯或牛肉蓋飯':0.08293168747068817, 
#                   '蕎麥麵或烏龍麵':0.0438939633458576, '日式蒲燒鰻或烤肉':0.11219844364240479, 
#                   '其他':0.0}

In [14]:
# for test
# print("math.floor(0.19565745694974226) : ", math.floor(0.19565745694974226))

In [ ]:
app = Flask(__name__)


@app.route("/",methods=["POST"])
def recommend():
    
    if flask.request.method == "POST":
        userdata = flask.request.json
        
    print('userdata:\n', userdata, '\n') 
     
    cls = clf.predict(np.array([userdata['age'], userdata['been2tko'],
                                userdata['days'], userdata['totalBudget'], 
                                userdata['gender'], userdata['oneBudget']]).reshape((1,-1)))[0]
    print('cls:\n', cls, '\n')    
    
    qryResult = []
    
#     distribution_0 = {'壽司或生魚片':0.16, '天婦羅炸雞或炸豬排':0.07, '壽喜燒或涮涮鍋':0.07, 
#                       '拉麵或日式炒麵':0.16, '和牛或河豚或懷石料理':0.16, '咖哩飯或牛肉蓋飯':0.06, 
#                       '蕎麥麵或烏龍麵':0.1, '日式蒲燒鰻或烤肉':0.13, '其他':0.0}
    
#     distribution_1 = {'壽司或生魚片':0.19, '天婦羅炸雞或炸豬排':0.12, '壽喜燒或涮涮鍋':0.08, 
#                       '拉麵或日式炒麵':0.2, '和牛或河豚或懷石料理':0.16, '咖哩飯或牛肉蓋飯':0.06, 
#                       '蕎麥麵或烏龍麵':0.09, '日式蒲燒鰻或烤肉':0.09, '其他':0.01}
    
#     distribution_2 = {'壽司或生魚片':0.2, '天婦羅炸雞或炸豬排':0.12, '壽喜燒或涮涮鍋':0.07, 
#                   '拉麵或日式炒麵':0.22, '和牛或河豚或懷石料理':0.15, '咖哩飯或牛肉蓋飯':0.08, 
#                   '蕎麥麵或烏龍麵':0.04, '日式蒲燒鰻或烤肉':0.11, '其他':0.0}

#     distribution_0 = {'壽司或生魚片':0.1746031746031746, '天婦羅炸雞或炸豬排':0.07936507936507936, 
#                       '壽喜燒或涮涮鍋':0.07936507936507936, '拉麵或日式炒麵':0.1746031746031746, 
#                       '和牛或河豚或懷石料理':0.1746031746031746, '咖哩飯或牛肉蓋飯':0.06349206349206349, 
#                       '蕎麥麵或烏龍麵':0.1111111111111111, '日式蒲燒鰻或烤肉':0.14285714285714285, 
#                       '其他':0.0}
    
#     distribution_1 = {'壽司或生魚片':0.19131148442456664, '天婦羅炸雞或炸豬排':0.1174003870576347, 
#                       '壽喜燒或涮涮鍋':0.1608654339084011, '拉麵或日式炒麵':0.19565745694974226, 
#                       '和牛或河豚或懷石料理':0.1746031746031746, '咖哩飯或牛肉蓋飯':0.06087317979140515, 
#                       '蕎麥麵或烏龍麵':0.08694901494245873, '日式蒲燒鰻或烤肉':0.09130208293298156, 
#                       '其他':0.013037917575526796}
    
#     distribution_2 = {'壽司或生魚片':0.20488392783360218, '天婦羅炸雞或炸豬排':0.11707668350520666, 
#                       '壽喜燒或涮涮鍋':0.06828247965477237, '拉麵或日式炒麵':0.21951864742200775, 
#                       '和牛或河豚或懷石料理':0.15121416712546054, '咖哩飯或牛肉蓋飯':0.08293168747068817, 
#                       '蕎麥麵或烏龍麵':0.0438939633458576, '日式蒲燒鰻或烤肉':0.11219844364240479, 
#                       '其他':0.0}
    
    distribution_0 = {'壽司或生魚片': int(0.1746031746031746 * 10), '天婦羅炸雞或炸豬排': int(0.07936507936507936 * 10), 
                      '壽喜燒或涮涮鍋': int(0.07936507936507936 * 10), '拉麵或日式炒麵': int(0.1746031746031746 * 10), 
                      '和牛或河豚或懷石料理': int(0.1746031746031746 * 10), '咖哩飯或牛肉蓋飯': int(0.06349206349206349 * 10), 
                      '蕎麥麵或烏龍麵': int(0.1111111111111111 * 10), '日式蒲燒鰻或烤肉': int(0.14285714285714285 * 10), 
                      '其他': int(0.0 * 10)}
    
    distribution_1 = {'壽司或生魚片': int(0.19131148442456664 * 10), '天婦羅炸雞或炸豬排': int(0.1174003870576347 * 10), 
                      '壽喜燒或涮涮鍋': int(0.1608654339084011 * 10), '拉麵或日式炒麵': int(0.19565745694974226 * 10), 
                      '和牛或河豚或懷石料理': int(0.1746031746031746 * 10), '咖哩飯或牛肉蓋飯': int(0.06087317979140515 * 10), 
                      '蕎麥麵或烏龍麵': int(0.08694901494245873 * 10), '日式蒲燒鰻或烤肉': int(0.09130208293298156 * 10), 
                      '其他': int(0.013037917575526796 * 10)}
    
    distribution_2 = {'壽司或生魚片': int(0.20488392783360218 * 10), '天婦羅炸雞或炸豬排': int(0.11707668350520666 * 10), 
                      '壽喜燒或涮涮鍋': int(0.06828247965477237 * 10), '拉麵或日式炒麵': int(0.21951864742200775 * 10), 
                      '和牛或河豚或懷石料理': int(0.15121416712546054 * 10), '咖哩飯或牛肉蓋飯': int(0.08293168747068817 * 10), 
                      '蕎麥麵或烏龍麵': int(0.0438939633458576 * 10), '日式蒲燒鰻或烤肉': int(0.11219844364240479 * 10), 
                      '其他': int(0.0 * 10)}    
    
    
    
    if cls == 0:
        distribution = distribution_0
    elif cls == 1:
        distribution = distribution_1
    else:
        distribution = distribution_2
###    
#     total = 0
#     for catg in list(distribution.keys()):
#         total += round(distribution[catg] * userdata['days'] * 2)
#         distribution[catg] = round(distribution[catg] * userdata['days'] * 2)
        
#     print('total:', total, '     distribution:', distribution)
    
#     for i in range(userdata['days'] * 2 - total):
#         distribution[int(distribution.keys())] += 1
###
 
        
    with conn.cursor() as cursor:
        usedb = "USE`test`"
        cursor.execute(usedb)
        
        print('distribution.keys(): \n', distribution.keys())
         
        for i, catg in enumerate(distribution.keys()):
            
            # 處理食物類型複合值的部份         
            for x, v_catg in enumerate(catg.split('或')):                
                if x == 0:
                    
                    sql_where = "CONCAT('%', '"+v_catg+"', '%') "
                    
                else:
                    sql_where += "OR CONCAT('%', '"+v_catg+"', '%') "                
        

            print('')
            print('distribution[catg]:', distribution[catg])
            print('天數:', userdata['days'])
            print('總預算:', userdata['totalBudget'])
            print('LIMIT: [',int(distribution[catg] * userdata['days']),']\n')
            
            
            sql = "SELECT * FROM tabelog " \
                  "WHERE rest_catg LIKE "+sql_where+"" \
                  "AND (prices_lunch BETWEEN 0 AND " + str(userdata['totalBudget']) + " " \
                  "OR prices_dinner BETWEEN 0 AND " + str(userdata['totalBudget']) + ") " \
                  "ORDER BY RAND() LIMIT " + str(int((distribution[catg] * userdata['days'] * 2) / 4))
         
          
            print('')
            print(i,' .sql:\n', sql, '\n')
            
            cursor.execute(sql) 
            qryResult += cursor.fetchall()

    
    
    print('')
    print('catg:', 'distribution[catg]', 'len(qryResult)')
    print(catg, distribution[catg], len(qryResult))
    print('\n\n')
    
    
    return flask.jsonify(qryResult)


#string, dict, tuple, Response instance
if __name__ == "__main__":
    clf = joblib.load('food_clf.joblib')
    
    app.run(port = 5009)

 * Running on http://127.0.0.1:5009/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Nov/2019 12:14:20] "POST / HTTP/1.1" 200 -


userdata:
 {'age': 35, 'been2tko': 20, 'days': 5, 'totalBudget': 80000, 'gender': 0, 'oneBudget': 400} 

cls:
 0 

distribution.keys(): 
 dict_keys(['壽司或生魚片', '天婦羅炸雞或炸豬排', '壽喜燒或涮涮鍋', '拉麵或日式炒麵', '和牛或河豚或懷石料理', '咖哩飯或牛肉蓋飯', '蕎麥麵或烏龍麵', '日式蒲燒鰻或烤肉', '其他'])

distribution[catg]: 1
天數: 5
總預算: 80000
LIMIT: [ 5 ]


0  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽司', '%') OR CONCAT('%', '生魚片', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 2 


distribution[catg]: 0
天數: 5
總預算: 80000
LIMIT: [ 0 ]


1  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '天婦羅炸雞', '%') OR CONCAT('%', '炸豬排', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 0 


distribution[catg]: 0
天數: 5
總預算: 80000
LIMIT: [ 0 ]


2  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽喜燒', '%') OR CONCAT('%', '涮涮鍋', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND

127.0.0.1 - - [20/Nov/2019 12:15:32] "POST / HTTP/1.1" 200 -


userdata:
 {'age': 35, 'been2tko': 20, 'days': 5, 'totalBudget': 80000, 'gender': 0, 'oneBudget': 300} 

cls:
 0 

distribution.keys(): 
 dict_keys(['壽司或生魚片', '天婦羅炸雞或炸豬排', '壽喜燒或涮涮鍋', '拉麵或日式炒麵', '和牛或河豚或懷石料理', '咖哩飯或牛肉蓋飯', '蕎麥麵或烏龍麵', '日式蒲燒鰻或烤肉', '其他'])

distribution[catg]: 1
天數: 5
總預算: 80000
LIMIT: [ 5 ]


0  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽司', '%') OR CONCAT('%', '生魚片', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 2 


distribution[catg]: 0
天數: 5
總預算: 80000
LIMIT: [ 0 ]


1  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '天婦羅炸雞', '%') OR CONCAT('%', '炸豬排', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 0 


distribution[catg]: 0
天數: 5
總預算: 80000
LIMIT: [ 0 ]


2  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽喜燒', '%') OR CONCAT('%', '涮涮鍋', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND

127.0.0.1 - - [20/Nov/2019 12:16:11] "POST / HTTP/1.1" 200 -


...

userdata:
 {'age': 35, 'been2tko': 20, 'days': 4, 'totalBudget': 80000, 'gender': 0, 'oneBudget': 300} 

cls:
 0 

distribution.keys(): 
 dict_keys(['壽司或生魚片', '天婦羅炸雞或炸豬排', '壽喜燒或涮涮鍋', '拉麵或日式炒麵', '和牛或河豚或懷石料理', '咖哩飯或牛肉蓋飯', '蕎麥麵或烏龍麵', '日式蒲燒鰻或烤肉', '其他'])

distribution[catg]: 1
天數: 4
總預算: 80000
LIMIT: [ 4 ]


0  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽司', '%') OR CONCAT('%', '生魚片', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 2 


distribution[catg]: 0
天數: 4
總預算: 80000
LIMIT: [ 0 ]


1  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '天婦羅炸雞', '%') OR CONCAT('%', '炸豬排', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 0 


distribution[catg]: 0
天數: 4
總預算: 80000
LIMIT: [ 0 ]


2  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽喜燒', '%') OR CONCAT('%', '涮涮鍋', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND

127.0.0.1 - - [20/Nov/2019 13:03:28] "POST / HTTP/1.1" 200 -


userdata:
 {'age': 35, 'been2tko': 20, 'days': 4, 'totalBudget': 80000, 'gender': 0, 'oneBudget': 300} 

cls:
 0 

distribution.keys(): 
 dict_keys(['壽司或生魚片', '天婦羅炸雞或炸豬排', '壽喜燒或涮涮鍋', '拉麵或日式炒麵', '和牛或河豚或懷石料理', '咖哩飯或牛肉蓋飯', '蕎麥麵或烏龍麵', '日式蒲燒鰻或烤肉', '其他'])

distribution[catg]: 1
天數: 4
總預算: 80000
LIMIT: [ 4 ]


0  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽司', '%') OR CONCAT('%', '生魚片', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 2 


distribution[catg]: 0
天數: 4
總預算: 80000
LIMIT: [ 0 ]


1  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '天婦羅炸雞', '%') OR CONCAT('%', '炸豬排', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 0 


distribution[catg]: 0
天數: 4
總預算: 80000
LIMIT: [ 0 ]


2  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽喜燒', '%') OR CONCAT('%', '涮涮鍋', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND

127.0.0.1 - - [20/Nov/2019 13:14:26] "POST / HTTP/1.1" 200 -


userdata:
 {'age': 35, 'been2tko': 20, 'days': 4, 'totalBudget': 80000, 'gender': 0, 'oneBudget': 300} 

cls:
 0 

distribution.keys(): 
 dict_keys(['壽司或生魚片', '天婦羅炸雞或炸豬排', '壽喜燒或涮涮鍋', '拉麵或日式炒麵', '和牛或河豚或懷石料理', '咖哩飯或牛肉蓋飯', '蕎麥麵或烏龍麵', '日式蒲燒鰻或烤肉', '其他'])

distribution[catg]: 1
天數: 4
總預算: 80000
LIMIT: [ 4 ]


0  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽司', '%') OR CONCAT('%', '生魚片', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 2 


distribution[catg]: 0
天數: 4
總預算: 80000
LIMIT: [ 0 ]


1  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '天婦羅炸雞', '%') OR CONCAT('%', '炸豬排', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND 80000) ORDER BY RAND() LIMIT 0 


distribution[catg]: 0
天數: 4
總預算: 80000
LIMIT: [ 0 ]


2  .sql:
 SELECT * FROM tabelog WHERE rest_catg LIKE CONCAT('%', '壽喜燒', '%') OR CONCAT('%', '涮涮鍋', '%') AND (prices_lunch BETWEEN 0 AND 80000 OR prices_dinner BETWEEN 0 AND

In [ ]:
cursor.close()
conn.close()